# Data Analysis

## Outline

- [Data Visualization](#Data-Visualization)
- [Targets and Features](#Targets-and-Features)
    - [item_id](#item_id)
    - [name](#name)
    - [category](#category)
    - [price](#price)
    - [old-price](#old_price)
    - [sellable online](#sellable_online)
    - [other colors](#other_colors)
    - [designer](#designer)
    - [height,width,depth](#hight,-width,-depth)
    - [link, short_description](#link,-short_description)
- [Correlation](#Correlation)
- [Data Cleaning](#Data-Cleaning)
    - [NaN in size data](#NaN-in-size-data)
- [Save final Dataframe](#Save-final-Dataframe)

## Data Visualization

In [ ]:
import pandas as pd
import numpy as np
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(style="white")


%matplotlib inline
plt.style.use('ggplot') # emulate pretty r-style plots

In [ ]:
#read the data and delete the index of entry
IKEA_df = pd.read_csv('Data/IKEA_SA_Furniture_Web_Scrapings_sss.csv')
IKEA_df = IKEA_df.drop(IKEA_df.columns[0],axis =1)

In [ ]:
IKEA_df.head(2)

In [62]:
m = IKEA_df.shape[0]
n = IKEA_df.shape[1]
print('We have '+str(m)+' sample sand '+str(n-1)+' features together with the target category')

We have 3694 sample sand 12 features together with the target category


data type of 12 features together with target category are shown as follows:

In [ ]:
# data type  
IKEA_df.dtypes

We can see that some data is missing in feature depth, height and width 

In [ ]:
IKEA_df.count() 

# wouldn't it be prettier if we only print the features for which that is the case instead of all of them?
# simply a suggestion...

max_count = max(IKEA_df.count())

missing_data = []
for i in range(len(IKEA_df.count())):
        if IKEA_df.count()[i] < max_count:
            missing_data.append(i)
            
IKEA_df.count()[missing_data]

## Target and features
Now we start to examine the properties of data for each features, which help us better understand the dataset and gives us an ideas of how to clean the dataset.

### item_id

In [ ]:
ax0 = IKEA_df.boxplot(column='item_id',by='category',vert=False, figsize=(10,6), fontsize =14,showmeans =True)
ax0.set_xlabel('')
ax0.figure.savefig('Figures/Data_Analysis/item_id_boxplot.png',bbox_inches = 'tight')

In [ ]:
nun_id =  IKEA_df['item_id'].nunique()
print('There are ' + str(nun_id)+' unique item_ids.')
if nun_id <  m:
    print('item_id is not unique for samples, some samples share same id or might be same one.')

The item_id is a number consisting of 5 to 8 digits. In the boxplot above we can see, that approximately 50% of the items of most categories are taking a value between 2e^7 to 7e^7. This means that item_id might does not give strong evidence of category.   

### name

In [ ]:
nun =  IKEA_df['name'].nunique()
#print(IKEA_df['category'].unique())
print('There are ' + str(nun) + ' unique name in total' )

In [ ]:
name_df =  IKEA_df.groupby('category').agg({'name':'nunique', 'category':'size'})
name_df.columns=['n_uname','size']

In [ ]:
name_df['%'] = name_df['n_uname']/ name_df['size']
name_df.sort_values(['%'], ascending=False)

Generally speaking, feature **name** can help determine the catogory. For Trolleys it might not work properlly, since 70% data samples have different  **name**. One the other hand, name might play an very important role for prediction on Sofas & armchairs, scine only 97% of the samples do not have the unique **name**.

### category

In [ ]:
nun =  IKEA_df['category'].nunique()
#print(IKEA_df['category'].unique())
print('There are ' + str(nun) + ' categories in total' )

In [ ]:
type_df = IKEA_df.groupby('category').size().reset_index(name='size').sort_values(['size'], ascending=False) 
type_df

In [ ]:
ax1 = type_df.set_index('category').plot.barh(rot=0, title='Category distribution',color = 'b',figsize=(10,6), fontsize=14)
ax1.set_xlabel("Number of Samples")
ax1.set_ylabel('')
ax1.figure.savefig('Figures/Data_Analysis/category_distribution.png',bbox_inches = 'tight')


In [ ]:
ax = type_df.set_index('category').plot.pie(title='Category distribution', y='size',figsize=(10,10), autopct='%1.1f%%',fontsize=12)
ax.set_ylabel('')
ax.legend(bbox_to_anchor=(2,0.5), loc="right", fontsize=12)
ax.figure.savefig('Figures/Data_Analysis/category_plotpie.png',bbox_inches = 'tight')

From the graph we can observe that more than 16% of the data samples are from the category Tables & desks, followed by Bookcase & sheiving units with approx. 15% and Chairs with approx. 13%. On the other hand, samples of category Room dividers, Sideboards, buffets & console tables, Cafe furniture and Trolleys acount for less than 1%. From this we can conclude, that the classification task itself is quite imbalanced. Hence, it might be necessary to replicate samples of the categories, of which the sample size is extremely small.  

### price

In [ ]:
ax2 = IKEA_df.boxplot(column='price',by='category',vert=False, figsize=(12,6), fontsize =14,showmeans =True)
ax2.set_xlabel('Euro')
ax2.figure.savefig('Figures/Data_Analysis/price_boxplot.png',bbox_inches = 'tight')

As shown above in the boxplot of price, the furniture of the category Wardrobes are the most expensive, while the children's furnitures are the cheapest. Sofa & armchairshas has the biggest range in price from about 250€ to 70.000€ while the ourliers can reach up to 90.000€. Cafe furniture has the most stable price. The price area (Interquartile Range of price) of some categories do overlap, but in general it is fair to say that price can be a useful feature in order determine the category.

### old_price

In [ ]:
t = IKEA_df[IKEA_df['old_price'].str.contains('No old price')]['item_id'].count() /m
print( str( round(t,4)*100) + '% of feature old_price is no old price' )

Furthermore, it is  impossible to recover this data. It follows, that the feature old_price might not be helpful in order to determine the category. Therefore, it can be ignored for training. 

### sellable_online

In [ ]:
t = IKEA_df[IKEA_df['sellable_online'] == True]['item_id'].count()/m
print( str( round(t,4)*100) + '% of feature sellable_online is True' )


This indicates, that this feature cannot help to determine the category and therefore can be deleted for training.

### other_colors

In [ ]:
t = IKEA_df[IKEA_df['other_colors'].str.contains('Yes')]['item_id'].count()/m
print( str( round(t,4)*100) + '% of feature other_colors is Yes.' )


### designer

In [ ]:
nun_designer =  IKEA_df['designer'].nunique()
#print(IKEA_df['designer'].unique())
print('There are ' + str(nun_designer) + ' different designers in total' )

In [ ]:
designer_df = IKEA_df.groupby('designer').size().reset_index(name='count').sort_values(['count'], ascending=False)
designer_df['count%'] = round(designer_df['count'] *100/m ,2) 

In [ ]:
designer_df.head(5)

In [ ]:
t = designer_df[designer_df['count']< 2].shape[0]
print(str(t)+ ' of ' + str(nun_designer)+'designers design just 1 items in dataset')

In [ ]:
t = designer_df[designer_df['count']< 5].shape[0]
print(str(t)+ ' of ' + str(nun_designer)+' designers design less than 5 items in dataset')

In [ ]:
t  = designer_df[designer_df['count']< 10].shape[0]
print(str(t)+ ' of ' + str(nun_designer)+' designers design less than 10 items in dataset')

In [ ]:
IKEA_df['designer_len'] = IKEA_df['designer'].str.len() 
IKEA_df['designer_len'].describe()

In [ ]:
t = IKEA_df[IKEA_df['designer_len'] > 40].shape[0] 
print(str( round(t*100/m,2)) +'% samples have actually text of long description in feature designer' )

We find that some entries in desinger is actually a more detailed description of the furniture itself. We could replace them with "designer unknowm", however,  this might be misleading to the algorithm as it could be understood as if the furniture was designed by the same person when instead the designer is not known. 
Moreover we observe that furniture of the same category might have the same text with a long description for the feature **designer**. Considering those mistakes acount for 7% of the whole dataset, we can also just keep the original ones.

In [ ]:
IKEA_df[IKEA_df['designer_len'] > 40].head(2)

We now examine the favorite furniture category of the TOP3 designers:
1. IKEA of Schweden (22.41%)
2. Ehlén Johansson (4.36%)
3. Francis Cayouette (4.09%)

In [ ]:
df1 = IKEA_df[IKEA_df['designer'].str.contains('IKEA of Sweden')].groupby('category')['item_id'].count() \
                                    .reset_index(name = 'count')
axd1 = df1.set_index('category').plot.pie(title='IKEA of Sweden\'s favor of furniture category', y='count',figsize=(10,10), \
                                          autopct='%1.1f%%', fontsize=12, legend = None)
axd1.set_ylabel('')
#axd1.legend(bbox_to_anchor=(1.5,0.5), loc="right", fontsize=12)
axd1.figure.savefig('Figures/Data_Analysis/TOP1_designer.png',bbox_inches = 'tight')

IKEA of Sweden covers all 17  furniture catogeries and favor the Bookcases & shelving units.

In [ ]:
df2 = IKEA_df[IKEA_df['designer'].str.contains('Ehlén Johansson')].groupby('category')['item_id'].count() \
                                    .reset_index(name = 'count')
axd2 = df2.set_index('category').plot.pie(title='Ehlén Johansson\'s favor of furniture category', y='count',figsize=(10,10), \
                                          autopct='%1.1f%%', fontsize=12, legend = None)
axd2.set_ylabel('')
#axd2.legend(bbox_to_anchor=(1.5,0.5), loc="right", fontsize=12)
axd2.figure.savefig('Figures/Data_Analysis/TOP2_designer.png',bbox_inches = 'tight')

In [ ]:
t1 = IKEA_df[IKEA_df['category'].str.contains('Wardrobes')]['item_id'].count() 
t2 = IKEA_df[IKEA_df['category'].str.contains('Wardrobes') & IKEA_df['designer'].str.contains('Ehlén Johansson')]['item_id'].count() 
print(round(t2/t1, 4))

Designer Ehlén Johansson favors Wardrobes. And indeed, about 73% items of category Wardrobes are designed by Ehlén Johansson (including the cases IKEA of Sweden/Ehlén Johansson and Ehlén Johansson/IKEA of Sweden).

In [ ]:
df3 = IKEA_df[IKEA_df['designer'].str.contains('Francis Cayouette')].groupby('category')['item_id'].count() \
                                    .reset_index(name = 'count')
axd3 = df3.set_index('category').plot.pie(title='Francis Cayouette\'s favor of furniture category', y='count',figsize=(10,10),\
                                          autopct='%1.1f%%', fontsize=12, legend = None)
axd3.set_ylabel('')
#axd3.legend(bbox_to_anchor=(1.5,0.5), loc="right", fontsize=12)
axd3.figure.savefig('Figures/Data_Analysis/TOP3_designer.png',bbox_inches = 'tight')

In [ ]:
t1 = IKEA_df[IKEA_df['category'].str.contains('Sofas & armchairs')]['item_id'].count() 
t2 = IKEA_df[IKEA_df['category'].str.contains('Sofas & armchairs') & IKEA_df['designer'].str.contains('Francis Cayouette')]['item_id'].count() 
print(round(t2/t1, 4))

Designer Francis Cayouette favors Sofas & armchairs. In total, around 17% items of category Sofas & armchairs are designed by Francis Cayouette.

### depth, height, width

In [ ]:
ax3 = IKEA_df.boxplot(column='depth',by='category',vert=False, figsize=(10,6), fontsize =14,showmeans =True)
ax3.set_xlabel('cm')
ax3.figure.savefig('Figures/Data_Analysis/depth_boxplot.png',bbox_inches = 'tight')

In [ ]:
ax4 = IKEA_df.boxplot(column='height',by='category',vert=False, figsize=(10,6), fontsize =14,showmeans =True)
ax4.set_xlabel('cm')
ax4.figure.savefig('Figures/Data_Analysis/height_boxplot.png',bbox_inches = 'tight')


In [ ]:
ax5 = IKEA_df.boxplot(column='width',by='category',vert=False, figsize=(10,6), fontsize =14,showmeans =True)
ax5.set_xlabel('cm')
ax5.figure.savefig('Figures/Data_Analysis/width_boxplot.png',bbox_inches = 'tight')


Generally, the feature of size varies a lot for the different categories. This is the case in particular for the feature **depth**. Hence, the feature of size might play an important role in prediction of category. 

### link, short_description
Feature **link** and **short_desctiption** can be understood literally. Both feature **link** and **short_description** are different for the individual items and uncorrelated to the category. Therefore, **link** and **short_description** will be ignored for the training. However, it should not go unnoted that short description contains size data,  which are not scrapped correctly as feature **depth,height,width**. We can make use of this and recover the NaN in **depth,height,width**. Unfortunately, in most cases this is technically too compilicated. Furthermore, both feature **link** and **short_description** contain key words, which indicate  its category directly. It might be kind of cheating to make use of that, while at the same time it would be extremely difficult to do so.

## Correlation
We would like to check the colinerity of numerical (including boolean) features. We can encode the nominal features to integer values by using factorization. Although, this could be misleadings in some way, at this point we just want to have an idea of the correlation between features. In case of training an algorithm that does not understand nominal features, we might use the one-hot encoding to avoid misunderstandings caused by numeric values. However, this leads to quite high computational cost. For example, for 370 different designers, one-hot encoding will create 9 more features, respectively, taking value 0/1 to present the feature designer.

In [ ]:
corr_df = IKEA_df.copy()

In [ ]:
corr_df['category'] = pd.factorize(corr_df['category'])[0] + 1
corr_df['name'] = pd.factorize(corr_df['name'])[0] + 1
corr_df['sellable_online'] = pd.factorize(corr_df['sellable_online'])[0] + 1
corr_df['other_colors'] = pd.factorize(corr_df['other_colors'])[0] + 1
corr_df['designer'] = pd.factorize(corr_df['designer'])[0] + 1

In [ ]:
# correlation / scatter matrix here
corr = corr_df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(10, 6))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
ax.set_title('Correlation between features')
ax.figure.savefig('Figures/Data_Analysis/corr.png',bbox_inches = 'tight')

As shown in the analysis of the individual features above, **depth** and **price** play the relative more important role in determining the **category**, while **sellable_online** and **other_colors** is less correlated to **category**. 
The price of the items is relative high correlated with depth of items and the name of items is relative high correlated with designer, indicating the colinearity of the features, which might result in bad prediciton in certain algorithms. Anyways, the correlation between features (including the target category) are not at on a high level as all of them are below 0.3.

## Data Cleaning

In [ ]:
#convert object to boolean
IKEA_df['other_colors'] = IKEA_df['other_colors'].map({'Yes': True, 'No': False})

In [ ]:
IKEA_df[IKEA_df['designer'].str.contains('IKEA of Sweden/Ehlén Johansson')].shape

In [ ]:
IKEA_df[IKEA_df['designer'].str.contains('Ehlén Johansson/IKEA of Sweden')].shape

In [ ]:
# merging 'IKEA of Sweden/Ehlén Johansson' and 'Ehlén Johansson/IKEA of Sweden'
# we noticed when the designers that this is one and the same designer
IKEA_df['designer']= np.where(IKEA_df['designer'].str.contains('IKEA of Sweden/Ehlén Johansson'),  'Ehlén Johansson/IKEA of Sweden', IKEA_df.designer)


The following will let us make the conclusion, that any 'NaN' is in the row containing the data concerning regarding the item-size:

In [ ]:
# determine where data is missing (cell is NaN)
shape_NaN = IKEA_df[IKEA_df.isnull().any(axis=1)].shape 
IKEA_df[IKEA_df.isnull().any(axis=1)].head(4)

It seems like the data is only missing regarding item-size (depth, height, width). Let's check if that is the case:

In [ ]:
size = ['depth','height','width'] 
shape_size = IKEA_df[IKEA_df[size].isnull().any(axis=1)].shape 
if shape_size == shape_NaN:
    print('The assumption from above seems to be correct. The only data missing is regarding the item-size')

In [ ]:
IKEA_df[IKEA_df[IKEA_df.columns[~IKEA_df.columns.isin(size)]].isnull().any(axis=1)].shape

In [ ]:
IKEA_df[IKEA_df[size].isnull().all(axis=1)].head(4)

### NaN in size data

According to the observations made by comparing the given dataset to the product size shown on the website, we came to the conclusion, that there are several main reasons as to why data of the category size is missing \.
First of, it seems that some data is not successfully scraped from the website and not recorded in the short description either making it is impossible to recover the missing data. \
Second, some items represent a set of different furniture. Therefore there is no product size in that sense. \
Third, some items have variable size, for example, an items width width can vary from 190 cm up to 220 cm. Although some data is recorded in the short description, it is very difficult to extract the exact size for all items. The biggest problem is that it cannot be identified what numbers respond to depth, height and width, especially since that data in the short description often is incomplete.\
Last of, there are some small items where the measurements ar taken in milimeters instead of centimeter. Furthermore, for some items a item-diameter was recorded in the short descritpion instead of the item-height.


In [ ]:
#split description and size data in feature short description
IKEA_df['size']= IKEA_df['short_description'].str.rsplit(',', n=1).str[1]
IKEA_df['short_description']= IKEA_df['short_description'].str.rsplit(',', n=1).str[0]

In [ ]:
#split numbers and unit
IKEA_df['unit'] = IKEA_df['size'].str.replace('\d+|-|x', ' ')
IKEA_df['size_nounit'] = IKEA_df['size'].str.replace('cm|mm', '')

The following shows that indeed, **height** is not given for the small items. Thus, in this case it is convenient to extract the diameter from the short description representing the diameter and replacing the NaN in height with the recorded data from **short description**. Additionally, we find, that ccording to the link the size-data for small items, of which the diameter is measured with mm, is actually mm as well. Therefore this is corrected in the following.  

In [ ]:
IKEA_df[IKEA_df['unit'].str.contains('mm') & IKEA_df[size].isnull().any(axis=1)].shape

In [ ]:
IKEA_df[IKEA_df['unit'].str.contains('mm') & IKEA_df['height'].isnull()].shape

In [ ]:
IKEA_df['height'] = np.where(IKEA_df['unit'].str.contains('mm') , IKEA_df.size_nounit, IKEA_df.height)
IKEA_df['height'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.height.astype(float)/10, IKEA_df.height)
IKEA_df['height'] = IKEA_df['height'].astype(float)

In [ ]:
IKEA_df['depth'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.depth.astype(float)/10, IKEA_df.depth)
IKEA_df['width'] = np.where(IKEA_df['unit'].str.contains('mm'), IKEA_df.width.astype(float)/10, IKEA_df.width)

For simplicity, the NaN-cells in size-data is filled with either the mean or the median depending on the category and wether there are many outliers in size data (refer to the boxplot earlier). In case there are many outliers resulting in the mean beeing significantly higher than the median, the NaN-cells are filled with the median.

In [ ]:
#calculate median/mean of items of each furniture category 
dm =IKEA_df.groupby('category')['depth'].median().reset_index(name = 'depth_median')
dd = IKEA_df.groupby('category')['depth'].mean().reset_index(name = 'depth_mean')
hm = IKEA_df.groupby('category')['height'].mean().reset_index(name = 'height_mean')
hd = IKEA_df.groupby('category')['height'].median().reset_index(name = 'height_median')
wm= IKEA_df.groupby('category')['width'].median().reset_index(name = 'width_median')
wd =IKEA_df.groupby('category')['width'].mean().reset_index(name = 'width_mean')

In [ ]:
IKEA_df = IKEA_df.join(dm.set_index('category'), on='category')
IKEA_df = IKEA_df.join(dd.set_index('category'), on='category')
IKEA_df = IKEA_df.join(hm.set_index('category'), on='category')
IKEA_df = IKEA_df.join(hd.set_index('category'), on='category')
IKEA_df = IKEA_df.join(wm.set_index('category'), on='category')
IKEA_df = IKEA_df.join(wd.set_index('category'), on='category')

In [ ]:
IKEA_df['depth'] = np.where( (IKEA_df['depth'].isnull()) & (IKEA_df['category'].str.contains('Tables & desks')),  \
                              IKEA_df.depth_median, IKEA_df.depth )
IKEA_df['depth'] = np.where( (IKEA_df['depth'].isnull()) & (IKEA_df['category'].str.contains('Sofas & armchairs')),  \
                              IKEA_df.depth_median, IKEA_df.depth )        
IKEA_df['depth'] = np.where( (IKEA_df['depth'].isnull()) & (IKEA_df['category'].str.contains('Chairs')),  \
                              IKEA_df.depth_median, IKEA_df.depth )

In [ ]:
IKEA_df['height'] = np.where( (IKEA_df['height'].isnull()) & (IKEA_df['category'].str.contains('TV & median furniture')),  \
                              IKEA_df.height_median, IKEA_df.height )
IKEA_df['height'] = np.where( (IKEA_df['height'].isnull()) & (IKEA_df['category'].str.contains('Sideboards, buffets,console tables')),  \
                              IKEA_df.height_median, IKEA_df.height )

In [ ]:
IKEA_df['width'] = np.where((IKEA_df['width'].isnull()) & (IKEA_df['category'].str.contains('Wardrobes')),  \
                            IKEA_df.width_median, IKEA_df.width)
IKEA_df['width'] = np.where((IKEA_df['width'].isnull()) & (IKEA_df['category'].str.contains('Outdoor furniture')),  \
                            IKEA_df.width_median, IKEA_df.width)
IKEA_df['width'] = np.where((IKEA_df['width'].isnull()) & (IKEA_df['category'].str.contains('Carbinets & cupboards')),  \
                            IKEA_df.width_median, IKEA_df.width)
IKEA_df['width'] = np.where((IKEA_df['width'].isnull()) & (IKEA_df['category'].str.contains('Bookcases & shelving units')),  \
                            IKEA_df.width_median, IKEA_df.width)

In [ ]:
IKEA_df['depth'] = np.where(IKEA_df['depth'].isnull(), IKEA_df.depth_mean, IKEA_df.depth)
IKEA_df['height'] = np.where(IKEA_df['height'].isnull(), IKEA_df.height_mean, IKEA_df.height)
IKEA_df['width'] = np.where(IKEA_df['width'].isnull(), IKEA_df.width_mean, IKEA_df.width)

## Prediction with tree-based algorithm 
Tree-based algorithms do not work on calculating distance. Therefore it is sufficient to factorize the categorical features.

In [ ]:
learn = ['item_id','name','category','price','sellable_online','other_colors','designer','depth','height','width' ]
learn_df = IKEA_df [learn]

In [ ]:
#check if there are still any NaN-cells in the dataset
learn_df[learn_df.isna().any(axis =1)]['item_id'].count()

In [ ]:
#this cell gives warning about setting copy. this warining can be ignored in this project
#encode nominal feature into numeric ones
learn_df['category_num'] = pd.factorize(learn_df['category'])[0] + 1
learn_df['name'] = pd.factorize(learn_df['name'])[0] + 1
#learn_df['sellable_online'] = pd.factorize(learn_df['sellable_online'])[0] + 1
#learn_df['other_colors'] = pd.factorize(learn_df['other_colors'])[0] + 1
learn_df['designer'] = pd.factorize(learn_df['designer'])[0] + 1

In [ ]:
label = IKEA_df['category'].unique()
label = label.tolist()
print('labels:', label)

In [ ]:
t = learn_df.drop(columns = ['category','category_num']).shape[1]
print('There are '+str(t)+' features.')

After modifying and cleaning the data frame, it is suitable for tre-based algorithms.

## Save Dataframes

In [ ]:
# save IKEA_df
with open('Data/IKEA_df.p', 'wb') as df:   
    pickle.dump(IKEA_df, df)

# save learn_df
with open('Data/learn_df.p', 'wb') as df:   
    pickle.dump(learn_df, df)